In [27]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.4f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# 상가 면적 (pnu단위_19자리)
sql = 'select pnu, (min_price_1 + max_price_1)/2 as price_avg, base_ym from m2.cremao_price'
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df = pd.DataFrame(rows)
df.columns = ['pnu','price','base_ym']

# pnu별 주용도명
sql = '''
select main_purps_cd_nm, sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji as pnu
  from m1.bld_rgst
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_principal_use = pd.DataFrame(rows)
df_bld_principal_use.columns = ['principal_use','pnu']

# 주용도코드 붙이기
df = pd.merge(df, df_bld_principal_use, on='pnu', how='left')
df = df.drop_duplicates('pnu')
df = df[~df['principal_use'].isnull()].reset_index(drop=True)
df = df[['pnu', 'principal_use', 'price', 'base_ym']]


### 2. DB INSERT
import datetime

table = 'm2.datadam_price'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
     # drop & create
    cursor.execute(
    """
    drop table if exists m2.datadam_price;
    create table m2.datadam_price (    
    pnu varchar(19)
    , principal_use varchar(200)
    , price double precision
    , base_ym varchar(6)
    )
    distkey(pnu)
    compound sortkey(base_ym);
    commit;
    """
    )
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-11-01 19:30:25.359583  :  0
2022-11-01 19:30:31.369312  :  10000
2022-11-01 19:30:33.804359  :  20000
2022-11-01 19:30:36.938100  :  30000
2022-11-01 19:30:41.621836  :  40000
2022-11-01 19:30:45.608987  :  50000
2022-11-01 19:30:48.520950  :  60000
2022-11-01 19:30:52.136512  :  70000
2022-11-01 19:30:58.102954  :  80000
2022-11-01 19:31:01.203289  :  90000
2022-11-01 19:31:03.610693  :  100000
2022-11-01 19:31:06.076874  :  110000
2022-11-01 19:31:08.545804  :  120000
2022-11-01 19:31:11.051346  :  130000
2022-11-01 19:31:13.732930  :  140000
2022-11-01 19:31:16.755485  :  150000
2022-11-01 19:31:20.793004  :  160000
2022-11-01 19:31:23.447387  :  170000
2022-11-01 19:31:25.999073  :  180000
2022-11-01 19:31:29.843626  :  190000
2022-11-01 19:31:32.477453  :  200000
2022-11-01 19:31:35.313335  :  210000
2022-11-01 19:31:38.349760  :  220000
2022-11-01 19:31:41.062521  :  230000
2022-11-01 19:31:44.181496  :  240000
2022-11-01 19:31:52.829691  :  250000
2022-11-01 19:31:56.627501

In [26]:
df

,pnu,principal_use,price,base_ym
0,4888036025108680000,단독주택,15499752.5331,202208
1,4790036042100320000,제2종근린생활시설,29780196.8047,202208
2,4790036042100700000,창고시설,41920298.3972,202208
3,4691025328105900006,창고시설,13576380.7126,202208
4,4691025328105950001,창고시설,15335610.9447,202208
...,...,...,...,...
1003155,2729011300102950002,제2종근린생활시설,36175258.1301,202208
1003156,2729011300102960003,종교시설,36400815.5427,202208
1003157,2729011300102960012,단독주택,38145201.7993,202208
1003158,2729011300102960015,단독주택,37616695.9770,202208


In [23]:
df.describe()

,price
count,1003160.0000
mean,35009645.9412
std,21322518.8935
min,4386228.6780
25%,22188232.7370
50%,31121523.5063
75%,42355426.9159
max,408048715.2277
